<a href="https://colab.research.google.com/github/Trandinhkhanh95/Complete-Python-3-Bootcamp/blob/master/MSc_IA_25_0002829EN_Khanh_Tran_Dinh_E1403_Final_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 0 - Imports & Config

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pathlib import Path
import re
CSV_PATH = "https://raw.githubusercontent.com/cpython-projects/E1402/refs/heads/main/course_sales.csv"
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")

In [2]:
!pip install ydata-profiling

# Step 1 - Data Loading & Inspect

In [3]:
df = pd.read_csv(CSV_PATH)
print("Shape:", df.shape)
print("\nColumns:", df.columns.tolist())
display(df.head(5))
display(df.tail(5))
df.info()
display(df.describe(include="all"))
display(df.dtypes)

Shape: (4510, 5)

Columns: ['Sale Date', 'Course Name', 'Price$', 'Quantity', 'Total']


,Sale Date,Course Name,Price$,Quantity,Total
0,"Jan 01, 2023",Python Basics,50,18.00,900
1,01.01.23,DS Boot,200,10.00,2000
2,"Jan 01, 2023",WEB DEVELOPMENT,80,16.00,1280
3,01/01/2023,Machine Learning ADV,250,11.00,2750
4,2023-01-01,SQL,60,13.00,780


,Sale Date,Course Name,Price$,Quantity,Total
4505,08/02/2025,ML (level 2),250,15.00,3750
4506,27.10.24,web_dev,80,15.00,1200
4507,"Jan 27, 2025",python for beginners,50,16.00,800
4508,20/03/2025,Data.Science,200,17.00,3400
4509,"Dec 09, 2024",Web Dev 101,80,20.00,1600


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4510 entries, 0 to 4509
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sale Date    4510 non-null   object 
 1   Course Name  4510 non-null   object 
 2   Price$       4510 non-null   int64  
 3   Quantity     4405 non-null   float64
 4   Total        4510 non-null   int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 176.3+ KB


,Sale Date,Course Name,Price$,Quantity,Total
count,4510,4510,"4,510.00","4,405.00","4,510.00"
unique,2969,20,NaN,NaN,NaN
top,"Dec 09, 2024",WEB DEVELOPMENT,NaN,NaN,NaN
freq,5,562,NaN,NaN,NaN
mean,NaN,NaN,128.02,12.92,"1,662.34"
std,NaN,NaN,81.33,7.95,"1,649.02"
min,NaN,NaN,50.00,0.00,0.00
25%,NaN,NaN,60.00,8.00,660.00
50%,NaN,NaN,80.00,13.00,"1,040.00"
75%,NaN,NaN,200.00,16.00,"2,600.00"


,0
Sale Date,object
Course Name,object
Price$,int64
Quantity,float64
Total,int64


# Step 2 - Data Standardization

In [4]:
# Step 2.0: Rename columns
df.columns = (
    df.columns
    .str.strip()                                # remove extra spaces
    .str.upper()                                # convert to uppercase
    .str.replace(r"[^\w\s]", "", regex=True)    # remove special characters
    .str.replace(" ", "_")                      # replace spaces with "_"
)

# Step 2.1: Standardize date columns with multiple formats
from datetime import datetime
fmts = ['%d.%m.%y','%d/%m/%Y','%Y-%m-%d','%d.%m.%Y','%b %d, %Y','%Y/%m/%d']
def parse_date(x):
    for f in fmts:
        try: return datetime.strptime(str(x), f)
        except: continue
    return pd.to_datetime(x, errors='coerce', dayfirst=True)

df["SALE_DATE"] = df["SALE_DATE"].apply(parse_date)
df = df.dropna(subset=["SALE_DATE"])
df = df[(df["SALE_DATE"] >= "2023-01-01") & (df["SALE_DATE"] <= "2025-05-31")]

# Step 2.2: Standardize numeric and categorical columns
for c in ["PRICE", "TOTAL"]:
    df[c] = (df[c].astype(str)
                   .str.replace(r"[^0-9.\-]", "", regex=True)
                   .replace({"": np.nan, ".": np.nan}))
    df[c] = pd.to_numeric(df[c], errors="coerce")

df["QUANTITY"] = pd.to_numeric(df["QUANTITY"], errors="coerce").astype("Int64")
df["COURSE_NAME"] = df["COURSE_NAME"].astype(str).str.strip().str.title()
df["TOTAL"] = df["PRICE"] * df["QUANTITY"]

# Step 2.3: Remove rows with missing or invalid critical values
df = df.dropna(subset=["PRICE", "QUANTITY", "TOTAL"])
df = df[df["QUANTITY"] >= 0]
df = df[df["PRICE"] >= 0]

# Step 2.4: Remove duplicates
dup = df.duplicated().sum()
df = df.drop_duplicates()
print(f"Removed duplicates: {dup}")


# Step 2.5: Detect outliers in QUANTITY using the IQR method
Q1, Q3 = df["QUANTITY"].quantile([0.25, 0.75])   # 25th and 75th percentile
IQR = Q3 - Q1
lower_bound = Q1 - 3 * IQR
upper_bound = Q3 + 3 * IQR
df["IS_OUTLIER_QUANTITY"] = (df["QUANTITY"] < lower_bound) | (df["QUANTITY"] > upper_bound) # Flag outliers

# Count outliers before removing
outlier_count = df["IS_OUTLIER_QUANTITY"].sum()
outlier_rate = (outlier_count / len(df)) * 100
print(f"Outlier count before removal: {outlier_count} ({outlier_rate:.2f}%)")

# Remove outliers
df = df[(df['QUANTITY'] >= lower_bound) & (df['QUANTITY'] <= upper_bound)]

Removed duplicates: 95
Outlier count before removal: 32 (0.74%)


# Step 3 - Perform EDA with visualizations

In [5]:
from ydata_profiling import ProfileReport
profile = ProfileReport(df, title="Sales Data Profiling Report", explorative=True)
profile.to_file("sales_data_report.html")
print(" Final report: sales_data_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 6/6 [00:00<00:00, 61.94it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

 Final report: sales_data_report.html


In [6]:
from google.colab import files
files.download("sales_data_report.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
insights = (df.groupby("COURSE_NAME")
              .agg(Total_Sales=("TOTAL","sum"),
                   Total_Quantity=("QUANTITY","sum"),
                   Average_Price=("PRICE","mean"),
                   Transactions=("COURSE_NAME","count"))
              .sort_values("Total_Sales", ascending=False)
              .reset_index())
display(insights.head(10))

,COURSE_NAME,Total_Sales,Total_Quantity,Average_Price,Transactions
0,Ml (Level 2),1106000,4424,250.00,360
1,Data Science Bootcamp,840400,4202,200.00,339
2,Ml+,545000,2180,250.00,167
3,Machine Learning Advanced,534250,2137,250.00,166
4,Web Development,533520,6669,80.00,536
5,Machine Learning Adv,529000,2116,250.00,167
6,Bootcamp Data Science,436000,2180,200.00,176
7,Data.Science,415200,2076,200.00,172
8,Ds Boot,393000,1965,200.00,163
9,Sql Basics,253140,4219,60.00,335


In [12]:
df.to_csv("course_sales_clean.csv", index=False, encoding="utf-8-sig")
print("✅ File đã được lưu: course_sales_clean.csv")
from google.colab import files
files.download("course_sales_clean.csv")

✅ File đã được lưu: course_sales_clean.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
# 3.1 Distribution of Price and Quantity
fig = px.histogram(df, x="PRICE", nbins=30, title="Table 1: Distribution of Course Prices")
fig.show()

fig = px.box(df, y="QUANTITY", title="Table 2: Boxplot of Quantities Sold")
fig.show()

# Step 3.2. Total Quatity by month

df_monthly_quantity = df.groupby(df["SALE_DATE"].dt.to_period("M")).agg(
    TOTAL_QUANTITY=("QUANTITY", "sum")
).reset_index()
df_monthly_quantity["SALE_DATE"] = df_monthly_quantity["SALE_DATE"].dt.to_timestamp()
fig = px.bar(df_monthly_quantity,
             x="SALE_DATE",
             y="TOTAL_QUANTITY",
             title="Table 3: Monthly Quantity of Courses Sold",
             labels={"SALE_DATE": "Month", "TOTAL_QUANTITY": "Total Quantity"},
             text="TOTAL_QUANTITY")
fig.update_traces(textposition="outside")
fig.show()


# 3.3. Sales trend over time
df_monthly = df.groupby(df["SALE_DATE"].dt.to_period("M")).agg(
    TOTAL_SALES=("TOTAL", "sum"),
    TOTAL_QUANTITY=("QUANTITY", "sum")
).reset_index()
df_monthly["SALE_DATE"] = df_monthly["SALE_DATE"].dt.to_timestamp()

fig = px.line(df_monthly, x="SALE_DATE", y="TOTAL_SALES", title="Table 4: Monthly Sales Trend")
fig.show()


# Step 3.5: Top 10 selling courses (quantity) ===
top10 = df.groupby("COURSE_NAME")["QUANTITY"].sum().nlargest(10).reset_index()
fig = px.bar(top10, x="COURSE_NAME", y="QUANTITY", title="Table 5: Top 10 Courses by Quantity")
fig.show()

# Step 3.6. Sales by course
course_sales = df.groupby("COURSE_NAME").agg(
    TOTAL_SALES=("TOTAL", "sum"),
    QUANTITY=("QUANTITY", "sum")
).sort_values("TOTAL_SALES", ascending=False)

fig = px.bar(course_sales.reset_index(),
             x="COURSE_NAME", y="TOTAL_SALES",
             title="Table 6: Total Sales by Course",
             text_auto=True)
fig.show()

# 3.7: Largest Drop in Quantity (Apr → May 2025) — All Courses

df["_YEAR_MONTH"] = df["SALE_DATE"].dt.to_period("M").astype(str)
# Filer data from Apr–May 2025
apr_may = df[df["_YEAR_MONTH"].isin(["2025-04", "2025-05"])]

# Total Quantity & Sales by course and month
course_month_stats = (
    apr_may.groupby(["COURSE_NAME", "_YEAR_MONTH"], as_index=False)
           .agg(TOTAL_QUANTITY=("QUANTITY", "sum"),
                TOTAL_SALES=("TOTAL", "sum"))
)
# Pivot to compare Apr–May
pivot_qty = (
    course_month_stats
    .pivot(index="COURSE_NAME", columns="_YEAR_MONTH", values="TOTAL_QUANTITY")
    .reindex(columns=["2025-04", "2025-05"])
    .fillna(0)
)
pivot_sales = (
    course_month_stats
    .pivot(index="COURSE_NAME", columns="_YEAR_MONTH", values="TOTAL_SALES")
    .reindex(columns=["2025-04", "2025-05"])
    .fillna(0)
)
# Diference
pivot_qty["CHANGE_QUANTITY"] = pivot_qty["2025-05"] - pivot_qty["2025-04"]
pivot_sales["CHANGE_SALES"] = pivot_sales["2025-05"] - pivot_sales["2025-04"]

merged_df = pivot_qty[["CHANGE_QUANTITY"]].merge(
    pivot_sales[["CHANGE_SALES"]],
    left_index=True, right_index=True
).reset_index()

# Top 10 giảm nhiều nhất theo Quantity
top_drops_qty = merged_df.nsmallest(10, "CHANGE_QUANTITY")

# Top 10 giảm nhiều nhất theo Sales
top_drops_sales = merged_df.nsmallest(10, "CHANGE_SALES")

fig1 = px.bar(
    top_drops_qty,
    x="COURSE_NAME",
    y="CHANGE_QUANTITY",
    title="Table 7a: Largest Drop in Quantity (Apr → May 2025) — Top 10 Courses",
    labels={"COURSE_NAME": "Course", "CHANGE_QUANTITY": "Quantity Change (May - Apr)"},
    text="CHANGE_QUANTITY"
)
fig1.update_traces(textposition="outside")
fig1.update_layout(xaxis_tickangle=45, yaxis=dict(zeroline=True))
fig1.show()

fig2 = px.bar(
    top_drops_sales,
    x="COURSE_NAME",
    y="CHANGE_SALES",
    title="Table 7b: Largest Drop in Sales (Apr → May 2025) — Top 10 Courses",
    labels={"COURSE_NAME": "Course", "CHANGE_SALES": "Sales Change (May - Apr)"},
    text="CHANGE_SALES"
)
fig2.update_traces(textposition="outside")
fig2.update_layout(xaxis_tickangle=45, yaxis=dict(zeroline=True))
fig2.show()

# Step 4: Business insights


**Key Business Insights**

**1. Pricing is highly fixed and rigid (*Table 1*).**

All courses are clustered around just five price points: **50, 60, 80, 200, 250**. This simplifies pricing but leaves no middle-ground offers. Customers either buy at low entry points or pay premium tiers, which could be limiting opportunities to capture demand in the mid-price range.

**2. Most courses sell modestly, but a few outperform (*Table 2*).**

The majority of courses sell between **8–16 units**, with a **median of 13 units**. However, there are outliers reaching **30–35 units**, showing that a few standout courses carry much stronger demand compared to the rest.

**3. Strong seasonality in course sales volume (*Table 3*)**

Sales consistently **peak at the start and end of the year**—for example, January 2023 (2,221 units) and January 2024 (2,331 units). In contrast, June–July months fall below 800 units, revealing a mid-year slump when demand slows significantly.

**4. Revenue follows the same seasonal pattern (*Table 4*).**

Monthly revenue stays strong at around 260k–310k, but drops sharply in June and July of both 2023 and 2024. The dip in sales quantity directly drives revenue down, confirming that seasonality is a major factor impacting overall business performance.

**5. Web Development dominates in volume.**
The Web Development course is the clear leader, with over 6,700 units sold—far exceeding other courses. This indicates strong, consistent demand for entry-level or broad-skill training, making it the primary driver of student enrollments.

**6. Beginner courses generate large enrollments but limited revenue impact..**

Courses such as Python for Beginners (≈4,200 units) and Python Basics (≈4,000 units) attract substantial numbers of learners, placing them among the top-selling programs by volume. However, due to their relatively low price points, they contribute far less revenue compared to premium offerings like MI (Level 2) or Data Science Bootcamp. This highlights a gap between popularity and profitability, with beginner courses acting more as entry-level feeders rather than core revenue drivers.

**7. Web Development sees the biggest drop in students**.

Between April and May 2025, the Web Development course lost more than 240 enrollments – the highest decline among all programs. However, because it is priced lower than advanced courses, the revenue impact was relatively smaller.

**8. Revenue hit hardest in premium courses.**

Although their student numbers fell less, high-priced programs like MI (Level 2) and Data Science Bootcamp caused the largest revenue losses (over $74,000 combined). This shows that even a small drop in enrollments for premium courses can significantly affect total revenue.


In [10]:
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.8 MB/s eta 0:00:00
